# Predicting liver disease in Hepatitis C patients via blood analysis

Brief description: The identification of a disease stage could be very complex, sometimes involving the necessity of invasive measurements. In this context, the use of key metabolic markers has been recently described as a valuable tool for non invasive, but very efficient disease diagnostics. The present application concerns the development of a machine learning tool for the classification of end-stage liver disease in patients affected by Hepatitis C. The reference database concerns a cohort of both female and male individuals. The dataset contains several metabolic predictors (e.g., serum albumin level, gamma glutamyl transferase etc.) and one categorical outcome corresponding to the stage of the disease (e.g., hepatitis, fibrosis, cirrhosis,etc.). Based on the predictors, the main aim of the model is to correctly classify the stage of liver disease progression in the patients’ cohort.

In [347]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

Import dataset from the UCI Machine Learning repository
https://archive.ics.uci.edu/ml/datasets/HCV+data

In [348]:
df = pd.read_csv('hcvdat0.csv')
df

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,611,3=Cirrhosis,62,f,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
611,612,3=Cirrhosis,64,f,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3
612,613,3=Cirrhosis,64,f,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0
613,614,3=Cirrhosis,46,f,33.0,NaN,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0


In [349]:
df.head(5)

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


Initial data preprocessing

I will now convert the "Sex" predictor into a numerical one. 
- Sex = 0 Male 
- Sex = 1 Female

In [350]:
df['Sex'].replace({'m':0, 'f':1}, inplace=True)
df

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,0,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,0,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,0,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,611,3=Cirrhosis,62,1,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
611,612,3=Cirrhosis,64,1,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3
612,613,3=Cirrhosis,64,1,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0
613,614,3=Cirrhosis,46,1,33.0,NaN,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0


In [351]:
df.info

<bound method DataFrame.info of      Unnamed: 0       Category  Age  Sex   ALB    ALP    ALT    AST   BIL  \
0             1  0=Blood Donor   32    0  38.5   52.5    7.7   22.1   7.5   
1             2  0=Blood Donor   32    0  38.5   70.3   18.0   24.7   3.9   
2             3  0=Blood Donor   32    0  46.9   74.7   36.2   52.6   6.1   
3             4  0=Blood Donor   32    0  43.2   52.0   30.6   22.6  18.9   
4             5  0=Blood Donor   32    0  39.2   74.1   32.6   24.8   9.6   
..          ...            ...  ...  ...   ...    ...    ...    ...   ...   
610         611    3=Cirrhosis   62    1  32.0  416.6    5.9  110.3  50.0   
611         612    3=Cirrhosis   64    1  24.0  102.8    2.9   44.4  20.0   
612         613    3=Cirrhosis   64    1  29.0   87.3    3.5   99.0  48.0   
613         614    3=Cirrhosis   46    1  33.0    NaN   39.0   62.0  20.0   
614         615    3=Cirrhosis   59    1  36.0    NaN  100.0   80.0  12.0   

       CHE  CHOL   CREA    GGT  PROT  
0   

Let's now check the presence on Nan instances in the dataset. 
If present, we would need to define a strategy to deal with missing values 


In [352]:
df.isnull().sum()
#We have a few missing values in the dataset. 
#The total number of missing instances is 31 which represents the 5.04% (max estimation) of the overal dataset. 
#Give the small percentage, I will delete the missing value instances

Unnamed: 0     0
Category       0
Age            0
Sex            0
ALB            1
ALP           18
ALT            1
AST            0
BIL            0
CHE            0
CHOL          10
CREA           0
GGT            0
PROT           1
dtype: int64

In [353]:
df = df.dropna(axis=0)

In [354]:
df.isnull().sum()
#Now we can see we have no longer missing values in the dataset

Unnamed: 0    0
Category      0
Age           0
Sex           0
ALB           0
ALP           0
ALT           0
AST           0
BIL           0
CHE           0
CHOL          0
CREA          0
GGT           0
PROT          0
dtype: int64

Let's now extrapolate the matrix of predictors and the outcome vector (y) from the full dataset

In [355]:
#let's now extrapolate the outcome variable from the full dataset
y = df.loc[:,'Category'].values
y

array(['0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '0=Blood Donor', '0=Blood Donor', '0=Blood Donor',
       '0=Blood Donor', '

In [356]:
#Let's now convert the outcome variable 'Category' into a numerical format

In [357]:
le = LabelEncoder()

In [358]:
y = le.fit_transform(y)
y
#len(y) #check if the length of the vector 'y' corresponds to the length of the full dataset

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [359]:
#in this way y contains only numerical values so that: 

# 0 = Blood donor
# 1 = Suspect Blood donor 
# 2 = Hepatitis 
# 3 = Fibrosis 
# 4 = Cirrhosis

In [360]:
X = df.drop(['Category', 'Unnamed: 0'], axis = 1).values
X

array([[ 32. ,   0. ,  38.5, ..., 106. ,  12.1,  69. ],
       [ 32. ,   0. ,  38.5, ...,  74. ,  15.6,  76.5],
       [ 32. ,   0. ,  46.9, ...,  86. ,  33.2,  79.3],
       ...,
       [ 62. ,   1. ,  32. , ...,  55.7, 650.9,  68.5],
       [ 64. ,   1. ,  24. , ...,  63. ,  35.9,  71.3],
       [ 64. ,   1. ,  29. , ...,  66.7,  64.2,  82. ]])

In [361]:
#Data in the predictors matrix are then normalized according to the min_max_scaler. 
#This is done since predictors where characterised by different scales in their values

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
X_norm= pd.DataFrame(X)
X_norm

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.166667,0.0,0.350669,0.101653,0.020962,0.036694,0.032181,0.367578,0.218447,0.091495,0.011757,0.580336
1,0.166667,0.0,0.350669,0.145571,0.052713,0.044990,0.014890,0.650434,0.408981,0.061619,0.017172,0.760192
2,0.166667,0.0,0.475483,0.156427,0.108816,0.134014,0.025456,0.494997,0.457524,0.072822,0.044400,0.827338
3,0.166667,0.0,0.420505,0.100419,0.091554,0.038290,0.086936,0.394263,0.401699,0.067221,0.045328,0.741007
4,0.166667,0.0,0.361070,0.154947,0.097719,0.045310,0.042267,0.515677,0.350728,0.063486,0.039295,0.573141
...,...,...,...,...,...,...,...,...,...,...,...,...
584,0.648148,1.0,0.283804,0.086603,0.043465,0.444799,0.034582,0.322882,0.309466,0.044814,0.069926,0.858513
585,0.666667,1.0,0.358098,0.098692,0.057645,0.878111,0.188280,0.290193,0.373786,0.119597,0.149443,0.616307
586,0.722222,1.0,0.254086,1.000000,0.015413,0.318124,0.236311,0.276851,0.591019,0.044534,1.000000,0.568345
587,0.759259,1.0,0.135215,0.225759,0.006165,0.107849,0.092219,0.008005,0.192961,0.051349,0.048577,0.635492


Once obtained the outcome variable (y) and the normalized predictor matrix (X) we can generate our training, validation and test set. 

The starting dataset is divided in the training, validation and test set according to the following proportions: 60%, 20%, 20%.

In [362]:
X_train, X_rem, y_train, y_rem = train_test_split(X_norm,y, train_size=0.6)


test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(353, 12)
(353,)
(118, 12)
(118,)
(118, 12)
(118,)


(None, None)

In [363]:
#initialization of decision tree classifier
dt = DecisionTreeClassifier()

In [364]:
#train of the decision tree classifier
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [365]:
#
y_test_pred=dt.predict(X_test)

In [366]:
train_score = dt.score(X_train, y_train)
valid_score = dt.score(X_valid, y_valid)

In [367]:
print('Decision tree train/valid accuracies:', train_score, valid_score)

Decision tree train/valid accuracies: 1.0 0.9067796610169492


In [368]:
#let's now check the overal score on the test set: 

test_score = dt.score(X_test, y_test)
print('Decision tree test accuracy:', test_score)

Decision tree test accuracy: 0.9152542372881356


Let's now repeat the same analysis after performing hyperparameter tuning. 

The hyperparameters involved in this case concerns: 

- tree depth (max_depth),
- the number of features to consider when looking for the best split (max_features),
- The minimum number of samples required to be at a leaf node (min_sample_leaf)
- The minimum number of samples required to split an internal node (min_sample_split). 

Hyperparameter optimization was carried out by traditional grid search methods analysing the whole possible combinations of hyperparameters. 

In [369]:
#Hyperparameter tuning with grid search

In [370]:
dt = DecisionTreeClassifier()

In [371]:
#let's introduce the hyperparametera for optimization and their possible values

parameters = {"max_depth":[1, 2, 3, 4, 5, 7, 10],  
              "min_samples_leaf": [1, 3, 6, 10, 20], 
              "min_samples_split": [1,3,5,7,9,11],
              "max_features":['auto', 'sqrt', 'log2']}

In [372]:
clf = GridSearchCV(dt, parameters, n_jobs= 1)

In [ ]:
clf.fit(X_train, y_train)

In [374]:
#identification of the best hyperparameter combination

print(clf.best_params_)

{'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 9}


In [375]:
depth = clf.best_params_['max_depth']
features = clf.best_params_['max_features']
sample_leaf = clf.best_params_['min_samples_leaf']
sample_split = clf.best_params_['min_samples_split']
print('The optimized hyperparameters are: \nmax_depth:',depth, '\nmax_features:', features, '\nmin_samples_leaf:', sample_leaf, '\nmin_samples_split:', sample_split )

The optimized hyperparameters are: 
max_depth: 7 
max_features: auto 
min_samples_leaf: 1 
min_samples_split: 9


In [376]:
#let's now train the new decision tree based on optimizes hyperparameters

In [377]:
dt = DecisionTreeClassifier(max_depth = depth, max_features= features, min_samples_leaf= sample_leaf, min_samples_split=sample_split)
dt.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


DecisionTreeClassifier(max_depth=7, max_features='auto', min_samples_split=9)

In [378]:
#y_train_pred=dt.predict(X_train)
#y_test_pred=dt.predict(X_test)
tree_train = dt.score(X_train, y_train)
valid_test = dt.score(X_valid, y_valid)
print('Decision tree train/valid accuracies:', tree_train, valid_test)

Decision tree train/valid accuracies: 0.9716713881019831 0.8983050847457628


Let's now check its overall accuracy on the test set.

In [379]:
#dt.fit(X_train, y_train)

In [380]:
test_score = dt.score(X_test,y_test)
print('Score on the test set equal to:', test_score)

Score on the test set equal to: 0.9576271186440678


In conclusion: 
-  Thanks to hyperparameter optimization, the overall decision tree structure reported a higher classification accuracy on the test set, compared to the absence of hyperparameter optimization

- Hyperparameter optimization was performed by grid search, which evaluated all the possible combinations of hyperparameter values and selected the best for the purpose of the optimization. Grid search optimization turned out to be an efficient and time efficient process given the relative low number of concerned hyperparameters. The choice of hyperparameter optimization strategy would need to be tuned according to the specific model to be used. More complex strategies may be adopted whenever dealing with more complex models, where hyperparameter may be of different types and may show intercorrelations

# Extra 

Some classes appear less densely represented in the dataset compared to other classes. For example, "Blood donor" appears more often compared to "Hepatitis", "fibrosis", and  "cirrhosis".
In case we would be interested in these last classes of hepatic disease, we may want to implement an oversampling strategy. For example let's consider the scenario where we may want to correctly classify patients with fibrosis. In this case, the overal classification would resume to a binary classification represented by "fibrosis vs all", where "all" includes "Blood donor", "suspect blood donor", "hepatitis" and "cirrhosis". 
Oversampling strategy may be applied through the stratified sampling algorithm shown below.  

## Stratified sampling algorithm

Category of interest:"fibrosis"

### TO DO:
1. Divide the available data into two sets (strata):
    - all samples of the class of interest (set A).
    - all other samples (set B)
2. Construct the training set:
    - randomly select 50 per cent of the samples in set A.
    - add equally many samples from set B.
3. Construct the validation set:
    - select the remaining 50 per cent of samples from set A.
    - add enough samples from set B so as to restore the original ratio from the overall data set.
    

Let's first analyse the case when no oversampling technique is applied

In [381]:
data = np.concatenate([X_norm, np.expand_dims(y, axis=1)], axis=1)
train, validation = sklearn.model_selection.train_test_split(data, test_size= 0.5, shuffle=True, random_state=1)
y_train = train[:,-1]
y_valid = validation[:,-1]
X_train = train[:,0:-1]
X_valid = validation[:,0:-1]

predictions = dt.predict(X_valid)
validation_ys = y_valid
print('% correct predictions:', (validation_ys == predictions).sum()/len(validation_ys))
print('% incorrect predictions:', (validation_ys != predictions).sum()/len(validation_ys))

indices = np.where(validation_ys == 3)
print('% correct predictions on class of interest:', (validation_ys[np.where(validation_ys == 3)] == 
                                                      predictions[np.where(validation_ys == 3)]).sum()
      /len(validation_ys[np.where(validation_ys == 3)]))

% correct predictions: 0.9423728813559322
% incorrect predictions: 0.0576271186440678
% correct predictions on class of interest: 0.7142857142857143


Let's now see how the accuracy on the class of intereset may change when applying oversampling

In [382]:
#the fibrosis group is evidenced by the outcome variable y=3

setA = np.concatenate([X_norm, np.expand_dims(y, axis=1)], axis=1)[np.where(y==3)] #fibrosis instances will be put in set A 
setB = np.concatenate([X_norm, np.expand_dims(y, axis=1)], axis=1)[np.where(y!=3)] #all other instances will be put in set B

# randomly select 50% of training samples in set A

setA_train, setA_validation = sklearn.model_selection.train_test_split(setA, 
                                                                       test_size=0.5, shuffle=True, random_state=1)
setB_train, setB_validation = sklearn.model_selection.train_test_split(setB, 
                                                                       test_size= 1-len(setA_train)/len(setB), shuffle=True, random_state=1)
train = np.concatenate([setA_train, setB_train])
validation = np.concatenate([setA_validation, setB_validation[:int((len(setA_validation)* len(setB))/len(setA)), :]])

In [383]:
y_train = train[:,-1]
y_valid = validation[:,-1]
X_train = train[:,0:-1]
X_valid = validation[:,0:-1]

In [384]:
dt.fit(X_train, y_train)
predictions = dt.predict(X_valid)
validation_ys = y_valid
#validation_ys = validation[:, 2]
print('% correct predictions:', (validation_ys == predictions).sum()/len(validation_ys))
print('% incorrect predictions:', (validation_ys != predictions).sum()/len(validation_ys))

indices = np.where(validation_ys == 3)
print('% correct predictions on class of interest:', (validation_ys[np.where(validation_ys == 3)] == 
                                                      predictions[np.where(validation_ys == 3)]).sum()
      /len(validation_ys[np.where(validation_ys == 3)]))

% correct predictions: 0.7993197278911565
% incorrect predictions: 0.20068027210884354
% correct predictions on class of interest: 0.8333333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


By implementing oversampling technique, we have achieved a higher accuracy on the target category (fibrosis, y=3), that was represented with fewer data instances compared to others in the dataset. 